# 📋 Section 16: Expected Document Lists (EDLs) API Testing

**Comprehensive testing of VeevaVault Expected Document Lists APIs**

Expected Document Lists (EDLs) help measure the completeness of clinical research projects by tracking expected vs. actual documents.

## 🎯 Testing Scope

### Core EDL Operations
- **Create Placeholder from EDL Item** - Convert EDL items into content placeholders
- **EDL Queries** - Query EDL data and items
- **Project Completeness** - Measure project completion status
- **EDL Templates** - Manage templates and milestone mappings

### Key Features
- ✅ **Project Tracking** - Monitor document completeness
- ✅ **Milestone Integration** - Connect EDLs to project milestones  
- ✅ **Placeholder Generation** - Create document placeholders
- ✅ **Template Management** - Work with EDL templates
- ✅ **Safe Operations** - Read-only discovery and safe actions

---

In [1]:
# 📦 Enhanced Testing Framework Setup
import sys
import os
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')

# Core imports for EDL testing
import time
import json
import pandas as pd
from datetime import datetime
from typing import Dict, Any, Optional, List
from dataclasses import dataclass

# VeevaVault imports  
from veevavault.client import VaultClient
from veevavault.services.authentication import AuthenticationService
from veevavault.services.queries import QueryService

@dataclass
class APITestResult:
    """Data class for storing API test results"""
    test_name: str
    success: bool
    duration: float
    response_data: Optional[Dict] = None
    error_msg: Optional[str] = None
    endpoint: Optional[str] = None
    method: str = "GET"
    status_code: Optional[int] = None

class EnhancedEDLAPITester:
    """Enhanced Expected Document Lists API Testing Framework"""
    
    def __init__(self, client: VaultClient):
        self.client = client
        self.query_service = None  # Will initialize after authentication
        self.results: List[APITestResult] = []
        self.start_time = time.time()
        
        # EDL-specific tracking
        self.edl_items_discovered = 0
        self.edl_templates_found = 0
        self.placeholders_available = False
        self.milestone_integration = False
        self.project_data_available = False
    
    def log_test(self, test_name: str, success: bool, duration: float, 
                 response_data: Optional[Dict] = None, error_msg: Optional[str] = None,
                 endpoint: Optional[str] = None, method: str = "GET", 
                 status_code: Optional[int] = None):
        """Log test result with enhanced formatting"""
        result = APITestResult(
            test_name, success, duration, response_data, 
            error_msg, endpoint, method, status_code
        )
        self.results.append(result)
        
        # Print result with emoji
        status_emoji = "✅" if success else "❌"
        test_num = len(self.results)
        
        if success:
            print(f"        {status_emoji} PASS [{test_num:02d}]: {test_name} ({duration:.2f}s)")
        else:
            print(f"        {status_emoji} FAIL [{test_num:02d}]: {test_name} ({duration:.2f}s)")
            if error_msg:
                print(f"        ❌ Error: {error_msg}")

    def initialize_query_service(self):
        """Initialize query service after authentication"""
        if not self.query_service:
            self.query_service = QueryService(self.client)

    def test_edl_discovery(self):
        """Discover EDL items and structure"""
        start_time = time.time()
        
        try:
            # Initialize query service if needed
            self.initialize_query_service()
            
            # Query EDL items using VQL through QueryService
            query = "SELECT id, name__v, status__v, expected_document_type__v FROM edl_item__v LIMIT 50"
            response = self.query_service.query(query)
            
            if response and response.get('responseStatus') == 'SUCCESS':
                edl_data = response.get('data', [])
                self.edl_items_discovered = len(edl_data)
                
                # Check if we have any EDL items
                if self.edl_items_discovered > 0:
                    print(f"        🎯 Found {self.edl_items_discovered} EDL items")
                    
                    # Sample first item details
                    first_item = edl_data[0]
                    print(f"        📋 Sample EDL: {first_item.get('name__v', 'Unknown')}")
                else:
                    print(f"        ℹ️ No EDL items found in vault")
                    
                self.log_test(
                    "Discover EDL Items", True, time.time() - start_time,
                    response_data=response, endpoint="/query", method="POST"
                )
            else:
                error_msg = response.get('errors', [{}])[0].get('message', 'Query failed') if response else 'No response received'
                self.log_test(
                    "Discover EDL Items", False, time.time() - start_time,
                    error_msg=error_msg, endpoint="/query", method="POST"
                )
                
        except Exception as e:
            self.log_test(
                "Discover EDL Items", False, time.time() - start_time,
                error_msg=str(e), endpoint="/query", method="POST"
            )

    def test_edl_templates(self):
        """Test EDL template discovery"""
        start_time = time.time()
        
        try:
            # Initialize query service if needed
            self.initialize_query_service()
            
            # Query EDL templates using VQL through QueryService
            query = "SELECT id, name__v, status__v FROM edl_template__v LIMIT 50"
            response = self.query_service.query(query)
            
            if response and response.get('responseStatus') == 'SUCCESS':
                template_data = response.get('data', [])
                self.edl_templates_found = len(template_data)
                
                print(f"        🎯 Found {self.edl_templates_found} EDL templates")
                if self.edl_templates_found > 0:
                    first_template = template_data[0]
                    print(f"        🗂 Sample template: {first_template.get('name__v', 'Unknown')}")
                    
                self.log_test(
                    "EDL Template Discovery", True, time.time() - start_time,
                    response_data=response, endpoint="/query", method="POST"
                )
            else:
                error_msg = response.get('errors', [{}])[0].get('message', 'Template query failed') if response else 'No response received'
                self.log_test(
                    "EDL Template Discovery", False, time.time() - start_time,
                    error_msg=error_msg, endpoint="/query", method="POST"
                )
                
        except Exception as e:
            self.log_test(
                "EDL Template Discovery", False, time.time() - start_time,
                error_msg=str(e), endpoint="/query", method="POST"
            )

    def test_placeholders_creation_capability(self):
        """Test ability to create placeholders from EDL items"""
        start_time = time.time()
        
        try:
            # Initialize query service if needed
            self.initialize_query_service()
            
            # Check if we can access placeholder creation APIs
            # This is typically a read-only capability check
            query = "SELECT id, name__v FROM document__v WHERE lifecycle__v = 'unclassified_document_lifecycle__v' LIMIT 5"
            response = self.query_service.query(query)
            
            if response and response.get('responseStatus') == 'SUCCESS':
                placeholder_docs = response.get('data', [])
                self.placeholders_available = True
                
                print(f"        🎯 Placeholder capability verified")
                print(f"        📄 Found {len(placeholder_docs)} potential placeholder documents")
                
                self.log_test(
                    "Placeholder Creation Capability", True, time.time() - start_time,
                    response_data=response, endpoint="/query", method="POST"
                )
            else:
                self.placeholders_available = False
                error_msg = response.get('errors', [{}])[0].get('message', 'Placeholder check failed') if response else 'No response received'
                self.log_test(
                    "Placeholder Creation Capability", False, time.time() - start_time,
                    error_msg=error_msg, endpoint="/query", method="POST"
                )
                
        except Exception as e:
            self.placeholders_available = False
            self.log_test(
                "Placeholder Creation Capability", False, time.time() - start_time,
                error_msg=str(e), endpoint="/query", method="POST"
            )

    def test_milestone_integration(self):
        """Test milestone integration capabilities"""
        start_time = time.time()
        
        try:
            # Initialize query service if needed
            self.initialize_query_service()
            
            # Check if milestones are integrated with EDL system
            query = "SELECT id, name__v FROM milestone__v LIMIT 10"
            response = self.query_service.query(query)
            
            if response and response.get('responseStatus') == 'SUCCESS':
                milestone_data = response.get('data', [])
                self.milestone_integration = len(milestone_data) > 0
                
                print(f"        🎯 Found {len(milestone_data)} milestones")
                if self.milestone_integration:
                    print(f"        📋 Milestone integration available")
                
                self.log_test(
                    "Milestone Integration Check", True, time.time() - start_time,
                    response_data=response, endpoint="/query", method="POST"
                )
            else:
                self.milestone_integration = False
                error_msg = response.get('errors', [{}])[0].get('message', 'Milestone check failed') if response else 'No response received'
                self.log_test(
                    "Milestone Integration Check", False, time.time() - start_time,
                    error_msg=error_msg, endpoint="/query", method="POST"
                )
                
        except Exception as e:
            self.milestone_integration = False
            self.log_test(
                "Milestone Integration Check", False, time.time() - start_time,
                error_msg=str(e), endpoint="/query", method="POST"
            )

    def test_project_completeness_tracking(self):
        """Test project completeness tracking capabilities"""
        start_time = time.time()
        
        try:
            # Initialize query service if needed
            self.initialize_query_service()
            
            # Query project completeness data
            query = "SELECT id, name__v FROM project__v LIMIT 10"
            response = self.query_service.query(query)
            
            if response and response.get('responseStatus') == 'SUCCESS':
                project_data = response.get('data', [])
                self.project_data_available = len(project_data) > 0
                
                print(f"        🎯 Found {len(project_data)} projects for completeness tracking")
                if self.project_data_available:
                    first_project = project_data[0]
                    print(f"        📊 Sample project: {first_project.get('name__v', 'Unknown')}")
                
                self.log_test(
                    "Project Completeness Tracking", True, time.time() - start_time,
                    response_data=response, endpoint="/query", method="POST"
                )
            else:
                error_msg = response.get('errors', [{}])[0].get('message', 'Project completeness query failed') if response else 'No response received'
                self.log_test(
                    "Project Completeness Tracking", False, time.time() - start_time,
                    error_msg=error_msg, endpoint="/query", method="POST"
                )
                
        except Exception as e:
            self.log_test(
                "Project Completeness Tracking", False, time.time() - start_time,
                error_msg=str(e), endpoint="/query", method="POST"
            )

    def test_placeholder_capability_check(self):
        """Test placeholder capability check (alias for consistency)"""
        return self.test_placeholders_creation_capability()

    def test_project_completeness(self):
        """Test project completeness (alias for consistency)"""
        return self.test_project_completeness_tracking()

    def run_comprehensive_test_suite(self):
        """Run all EDL API tests"""
        print("🚀 Starting Comprehensive EDL API Test Suite...")
        print("="*60)
        
        # Run all test methods
        self.test_edl_discovery()
        self.test_edl_templates()
        self.test_placeholders_creation_capability()  
        self.test_milestone_integration()
        self.test_project_completeness_tracking()
        
        # Generate summary
        total_tests = len(self.results)
        success_count = sum(1 for r in self.results if r.success)
        total_time = time.time() - self.start_time
        
        print(f"\n📊 EDL API Test Summary:")
        print(f"   🎯 Total Tests: {total_tests}")
        print(f"   ✅ Successful: {success_count}")
        print(f"   ❌ Failed: {total_tests - success_count}")
        print(f"   ⏱️ Total Time: {total_time:.2f}s")
        print(f"   📈 Success Rate: {(success_count/total_tests*100):.1f}%")
        
        # EDL-specific summary
        print(f"\n📋 EDL Discovery Results:")
        print(f"   📄 EDL Items Found: {self.edl_items_discovered}")
        print(f"   🗂 EDL Templates Found: {self.edl_templates_found}")
        print(f"   📄 Placeholder Capability: {'✅' if self.placeholders_available else '❌'}")
        print(f"   🎯 Milestone Integration: {'✅' if self.milestone_integration else '❌'}")
        print(f"   📊 Project Data Available: {'✅' if self.project_data_available else '❌'}")
        
        # Export summary for continuation
        summary_data = {
            'section': 16,
            'name': 'Expected Document Lists',
            'success_rate': f"{(success_count/total_tests*100):.1f}%",
            'total_tests': total_tests,
            'successful_tests': success_count,
            'total_time': f"{total_time:.2f}s",
            'edl_items_discovered': self.edl_items_discovered,
            'edl_templates_found': self.edl_templates_found,
            'placeholder_capability': self.placeholders_available,
            'milestone_integration': self.milestone_integration,
            'project_data_available': self.project_data_available
        }
        
        print(f"\n📦 Section summary exported for continuation: {summary_data['success_rate']} success rate")
        print("="*60)
        
        return summary_data

    def generate_summary(self):
        """Generate a comprehensive summary of all tests"""
        return self.run_comprehensive_test_suite()

print("✅ Enhanced EDL API Testing Framework loaded!")
print("📋 Ready for Expected Document Lists API comprehensive testing")
print("🎯 Features: EDL discovery, templates, project tracking, milestone integration")

✅ Enhanced EDL API Testing Framework loaded!
📋 Ready for Expected Document Lists API comprehensive testing
🎯 Features: EDL discovery, templates, project tracking, milestone integration


In [2]:
# 🔐 Load credentials and initialize client
exec(open('/Users/mp/Documents/Code/VeevaTools/veevatools/veevavault/test_credentials.py').read())

# Extract credentials from TEST_VAULTS
vault_config = TEST_VAULTS[DEFAULT_VAULT]
VAULT_URL = vault_config["URL"]
VAULT_DNS = VAULT_URL.replace("https://", "").replace("/", "")
USERNAME = vault_config["username"]
PASSWORD = vault_config["password"]

print("🔐 Credentials loaded successfully!")
print(f"🏢 Vault DNS: {VAULT_DNS}")
print(f"👤 Username: {USERNAME}")
print("✅ Ready for EDL API testing!")

🔐 Credentials loaded successfully!
🏢 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
✅ Ready for EDL API testing!


In [3]:
# 🚀 Initialize Enhanced EDL API Testing Framework
from veevavault.client.vault_client import VaultClient

# Create client instance
client = VaultClient()
client.vaultDNS = VAULT_DNS

# Initialize enhanced EDL tester
edl_tester = EnhancedEDLAPITester(client)

print("🚀 Enhanced EDL API Testing Framework initialized!")
print("📋 Ready to test Expected Document Lists APIs")
print("🎯 Framework features:")
print("   ✅ EDL item discovery and analysis")
print("   ✅ Template management testing")
print("   ✅ Project completeness tracking")
print("   ✅ Milestone integration verification")
print("   ✅ Placeholder creation capability check")
print("   ✅ Safe operations with vault integrity preservation")

🚀 Enhanced EDL API Testing Framework initialized!
📋 Ready to test Expected Document Lists APIs
🎯 Framework features:
   ✅ EDL item discovery and analysis
   ✅ Template management testing
   ✅ Project completeness tracking
   ✅ Milestone integration verification
   ✅ Placeholder creation capability check
   ✅ Safe operations with vault integrity preservation


In [4]:
# 🔑 Authenticate with VeevaVault
print("🔑 Authenticating with VeevaVault...")

start_time = time.time()
try:
    # Set the vault URL first
    client.vaultURL = VAULT_URL
    
    # Authenticate using the authentication service
    from veevavault.services.authentication import AuthenticationService
    
    auth_service = AuthenticationService(client)
    auth_response = auth_service.authenticate_with_username_password(
        username=USERNAME,
        password=PASSWORD,
        vaultDNS=VAULT_DNS
    )
    auth_duration = time.time() - start_time
    
    if auth_response and auth_response.get('responseStatus') == 'SUCCESS':
        vault_id = auth_response.get('vaultId')
        session_id = auth_response.get('sessionId')
        user_id = auth_response.get('userId')
        
        print(f"✅ Authentication successful!")
        print(f"🏢 Vault ID: {vault_id}")
        print(f"👤 User ID: {user_id}")
        print(f"🔐 Session established: {session_id[:20]}...")
        
        # Store for framework
        client.vault_id = vault_id
        client.username = USERNAME
        
        # Log successful authentication
        edl_tester.log_test(
            "Authentication", True, auth_duration,
            response_data={'vault_id': vault_id, 'user_id': user_id},
            endpoint="/auth", method="POST"
        )
        
    else:
        error_msg = f"Authentication failed - Response: {auth_response}"
        print(f"❌ {error_msg}")
        edl_tester.log_test(
            "Authentication", False, auth_duration,
            error_msg=error_msg, endpoint="/auth", method="POST"
        )
        
except Exception as e:
    auth_duration = time.time() - start_time
    error_msg = f"Authentication exception: {str(e)}"
    print(f"❌ {error_msg}")
    edl_tester.log_test(
        "Authentication", False, auth_duration,
        error_msg=error_msg, endpoint="/auth", method="POST"
    )

print("\n🎯 Ready to test Expected Document Lists APIs!")

🔑 Authenticating with VeevaVault...
✅ Authentication successful!
🏢 Vault ID: 92425
👤 User ID: 13454867
🔐 Session established: 976553B26EB9811B2FF1...
        ✅ PASS [01]: Authentication (0.38s)

🎯 Ready to test Expected Document Lists APIs!
✅ Authentication successful!
🏢 Vault ID: 92425
👤 User ID: 13454867
🔐 Session established: 976553B26EB9811B2FF1...
        ✅ PASS [01]: Authentication (0.38s)

🎯 Ready to test Expected Document Lists APIs!


In [5]:
# 📋 Test 1: EDL Objects Discovery & Analysis
print("🔍 Testing EDL Objects Discovery...")
print("📋 Discovering Expected Document List objects and structure")

# Initialize query service
from veevavault.services.queries import QueryService
query_service = QueryService(client)

# Initialize results tracking
edl_test_results = []
edl_objects_found = []

start_time = time.time()

try:
    # First, let's discover what EDL-related objects exist in this vault
    print("\n🎯 Test 1: Discovering available objects...")
    
    # Check for standard objects that might contain EDL data
    test_objects = [
        'edl_item__v',
        'edl_template__v', 
        'expected_document_list__v',
        'document_list__v',
        'project__v',
        'milestone__v'
    ]
    
    available_objects = []
    
    for obj in test_objects:
        try:
            # Test with a simple query to see if object exists
            test_query = f"SELECT id FROM {obj} LIMIT 1"
            response = query_service.query(test_query)
            
            if response and response.get('responseStatus') == 'SUCCESS':
                available_objects.append(obj)
                print(f"   ✅ Object available: {obj}")
            else:
                print(f"   ❌ Object not available: {obj}")
                
        except Exception as e:
            print(f"   ❌ Error testing {obj}: {str(e)}")
    
    print(f"\n📊 Object Discovery Summary:")
    print(f"   🎯 Objects tested: {len(test_objects)}")
    print(f"   ✅ Available objects: {len(available_objects)}")
    print(f"   📋 Available: {', '.join(available_objects) if available_objects else 'None'}")
    
    edl_test_results.append(("Object Discovery", True, time.time() - start_time))
    
except Exception as e:
    print(f"❌ Exception in object discovery: {str(e)}")
    edl_test_results.append(("Object Discovery", False, time.time() - start_time))

# Test available objects for actual data
if available_objects:
    print(f"\n🔍 Testing data availability in discovered objects...")
    
    for obj in available_objects:
        try:
            start_time = time.time()
            query = f"SELECT id, name__v FROM {obj} LIMIT 10"
            response = query_service.query(query)
            
            if response and response.get('responseStatus') == 'SUCCESS':
                data = response.get('data', [])
                print(f"   ✅ {obj}: {len(data)} records found")
                
                if data and len(data) > 0:
                    first_record = data[0]
                    print(f"      📋 Sample: {first_record.get('name__v', first_record.get('id', 'Unknown'))}")
                
                edl_test_results.append((f"Data Test - {obj}", True, time.time() - start_time))
            else:
                error_msg = response.get('errors', [{}])[0].get('message', 'Query failed') if response else 'No response'
                print(f"   ❌ {obj}: Query failed - {error_msg}")
                edl_test_results.append((f"Data Test - {obj}", False, time.time() - start_time))
                
        except Exception as e:
            print(f"   ❌ {obj}: Exception - {str(e)}")
            edl_test_results.append((f"Data Test - {obj}", False, time.time() - start_time))

# Summary
successful_tests = sum(1 for _, success, _ in edl_test_results if success)
total_tests = len(edl_test_results)

print(f"\n📊 EDL Discovery Summary:")
print(f"   🎯 Total Tests: {total_tests}")
print(f"   ✅ Successful: {successful_tests}")
print(f"   ❌ Failed: {total_tests - successful_tests}")
print(f"   📄 Available EDL Objects: {len(available_objects)}")
print(f"   📈 Success Rate: {(successful_tests/total_tests*100):.1f}%" if total_tests > 0 else "   📈 Success Rate: 0%")

print(f"\n✅ EDL object discovery testing completed!")

# Store results for framework
edl_objects_discovered = len(available_objects)
edl_objects_list = available_objects

🔍 Testing EDL Objects Discovery...
📋 Discovering Expected Document List objects and structure

🎯 Test 1: Discovering available objects...
Success: 0 records returned.
   ✅ Object available: edl_item__v
Success: 0 records returned.
   ✅ Object available: edl_item__v
Success: 0 records returned.
   ✅ Object available: edl_template__v
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': 'expected_document_list__v is not a queryable object; please refer to api documentation'}]}
   ❌ Object not available: expected_document_list__v
Success: 0 records returned.
   ✅ Object available: edl_template__v
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': 'expected_document_list__v is not a queryable object; please refer to api documentation'}]}
   ❌ Object not available: expected_document_list__v
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': 'document_list__v is not a queryable object; please refe

In [6]:
# 📄 Test 2: EDL Templates Discovery  
print("🔍 Testing EDL Templates Discovery...")
print("📄 Discovering EDL templates and configurations")

edl_tester.test_edl_templates()

print(f"\n✅ EDL templates discovery completed!")
print(f"📄 Templates found: {edl_tester.edl_templates_found}")

🔍 Testing EDL Templates Discovery...
📄 Discovering EDL templates and configurations
Success: 0 records returned.
        🎯 Found 0 EDL templates
        ✅ PASS [02]: EDL Template Discovery (0.19s)

✅ EDL templates discovery completed!
📄 Templates found: 0


In [7]:
# 📊 Test 3: Project Completeness Tracking
print("🔍 Testing Project Completeness Tracking...")
print("📊 Checking project completion metrics via EDLs")

edl_tester.test_project_completeness()

print(f"\n✅ Project completeness test completed!")
print(f"📊 Project data available: {edl_tester.project_data_available}")

🔍 Testing Project Completeness Tracking...
📊 Checking project completion metrics via EDLs
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': 'project__v is not a queryable object; please refer to api documentation'}]}
        ❌ FAIL [03]: Project Completeness Tracking (0.18s)
        ❌ Error: No response received

✅ Project completeness test completed!
📊 Project data available: False


In [8]:
# 🎯 Test 4: Milestone Integration
print("🔍 Testing EDL Milestone Integration...")
print("🎯 Checking milestone integration and mapping")

edl_tester.test_milestone_integration()

print(f"\n✅ Milestone integration test completed!")
print(f"🎯 Milestone integration: {edl_tester.milestone_integration}")

🔍 Testing EDL Milestone Integration...
🎯 Checking milestone integration and mapping
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': 'milestone__v is not a queryable object; please refer to api documentation'}]}
        ❌ FAIL [04]: Milestone Integration Check (0.18s)
        ❌ Error: No response received

✅ Milestone integration test completed!
🎯 Milestone integration: False


In [9]:
# 🔗 Test 5: Placeholder Creation Capability
print("🔍 Testing Placeholder Creation Capability...")
print("🔗 Checking placeholder creation capabilities (discovery only)")

edl_tester.test_placeholder_capability_check()

print(f"\n✅ Placeholder capability test completed!")
print(f"🔗 Placeholder creation available: {edl_tester.placeholders_available}")

🔍 Testing Placeholder Creation Capability...
🔗 Checking placeholder creation capabilities (discovery only)
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': 'document__v is not a queryable object; please refer to api documentation'}]}
        ❌ FAIL [05]: Placeholder Creation Capability (0.28s)
        ❌ Error: No response received

✅ Placeholder capability test completed!
🔗 Placeholder creation available: False
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': 'document__v is not a queryable object; please refer to api documentation'}]}
        ❌ FAIL [05]: Placeholder Creation Capability (0.28s)
        ❌ Error: No response received

✅ Placeholder capability test completed!
🔗 Placeholder creation available: False


In [10]:
# 📊 Generate Comprehensive Test Summary
print("📊 Generating comprehensive EDL API test summary...")

edl_summary = edl_tester.generate_summary()

# Store results for next section
section_16_results = {
    'edl_tester': edl_tester,
    'summary': edl_summary,
    'client': client
}

print(f"\n🎯 Section 16 (Expected Document Lists) completed!")
print(f"📋 Ready to continue to Section 17...")

📊 Generating comprehensive EDL API test summary...
🚀 Starting Comprehensive EDL API Test Suite...
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': "Unknown field 'expected_document_type__v' in 'select fields'"}]}
        ❌ FAIL [06]: Discover EDL Items (0.18s)
        ❌ Error: No response received
Success: 0 records returned.
        ❌ FAIL [07]: EDL Template Discovery (0.20s)
        ❌ Error: Template query failed
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': 'document__v is not a queryable object; please refer to api documentation'}]}
        ❌ FAIL [08]: Placeholder Creation Capability (0.18s)
        ❌ Error: No response received
Success: 0 records returned.
        ❌ FAIL [07]: EDL Template Discovery (0.20s)
        ❌ Error: Template query failed
Error: {'responseStatus': 'FAILURE', 'errors': [{'type': 'INVALID_DATA', 'message': 'document__v is not a queryable object; please refer to api documentation'}]}
   